In [1]:
import os
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd
from nltk.lm import Vocabulary
import pickle
import sys
import torch
import importlib
from timeit import default_timer as t
sys.path.append("../../lib")
from metrics import levenshtein
import ocr_correction
from pytorch_decoding import seq2seq

In [2]:
folder = "../../data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_evaluation_4M_without_Finnish/PL/"

In [3]:
files = sorted(os.listdir(folder))
len(files)

1

In [4]:
import glob

files = glob.glob(folder + '/**/*.txt', recursive=True)
len(files)

50

In [5]:
data = []
for f in tqdm(files):
    with open(f) as file:
        data.append(file.readlines())

  0%|          | 0/50 [00:00<?, ?it/s]

In [6]:
data = pd.DataFrame(data, 
                    columns = ["ocr_to_input", 
                               "ocr_aligned", 
                               "gs_aligned"])\
.assign(ocr_to_input = lambda df: df.ocr_to_input.str.replace("[OCR_toInput] ", "", regex = False),
        ocr_aligned = lambda df: df.ocr_aligned.str.replace("[OCR_aligned] ", "", regex = False),
        gs_aligned = lambda df: df.gs_aligned.str.replace("[ GS_aligned] ", "", regex = False))

print(data.shape)
data.head()

(50, 3)


,ocr_to_input,ocr_aligned,gs_aligned
0,&clacy<*. am h> pat <E>Uwfłi/o pultord jła 2fc...,@&clacy@@@@@@@@@@@@@@@@@<*.@@@@@@@ @a@@@@@@@@@...,Relácya. Bur⸗ grábiá nowy. Pierw⸗ ßacżɇṡċ ref...
1,47* EUROPA. O Konftantyncpoltt. chomet fam y B...,@47* EUROPA. O Konftantyncpoltt. chomet fam y ...,471 EUROPA. O Konstantynopol@u. chomet sam y ...
2,O Schizmie Aitgielfkiey Święcą Narodzenia Pańf...,@O Schizmie Aitgielfkiey Święcą Narodzenia Pań...,O Schizmie A@ngielskiey Swięta Národzenia Pań...
3,"Skrzydeł dobędzie z lerdeczney kaletki, Wody z...",@@@@@@@@@@@@@@Skrzydeł dobędzie z lerdeczney k...,Bb 3 Jdź 234 Skrzydeł dobędzie z serdeczney k...
4,tdt wOMfnSiech uiiSo wifcar yrftyey/ ińbztom v...,@@@@@t@@@@@dt w@@@@@@OMf@@@@nSi@@@e@@@@ch@@@@@...,To t Bibe w ocżⱥc ffe po miástecżkách y po...


In [7]:
data.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,50.000000,50.000000,50.000000
mean,1771.680000,1856.240000,1855.240000
std,551.119582,565.948168,565.948168
min,265.000000,276.000000,275.000000
25%,1587.000000,1653.000000,1652.000000
50%,1890.000000,1948.000000,1947.000000
75%,2008.500000,2081.250000,2080.250000
max,2802.000000,2875.000000,2874.000000


In [8]:
vocabulary = Vocabulary(data.ocr_to_input.sum() + data.ocr_aligned.sum() + data.gs_aligned.sum())
print(len(vocabulary))

177


In [9]:
distances = levenshtein(reference = data.gs_aligned.str.replace("@", ""), 
                        hypothesis = data.ocr_to_input)
distances.cer.describe()

count      50.000000
mean      191.952558
std      1167.803230
min         8.108108
25%        11.531529
50%        14.723677
75%        43.496655
max      8283.333333
Name: cer, dtype: float64

In [10]:
distances = levenshtein(reference = data.gs_aligned, 
                        hypothesis = data.ocr_to_input)
distances.cer.describe()

count    50.000000
mean     26.561643
std      20.048362
min       8.051118
25%      11.383476
50%      14.401381
75%      42.185983
max      98.954704
Name: cer, dtype: float64

In [11]:
data.to_pickle("../../data/pl/data/test.pkl")